In [2]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"


from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# uninstall
!pip uninstall -y wandb

# download
!pip install transformers

!pip install --upgrade accelerate

!pip install openai

!pip install python-dotenv

# import
import re
import json
import torch
import random
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import re
import json
import torch
import random
import os
import time
#import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset
#from sklearn.metrics import f1_score
#from sklearn.model_selection import train_test_split
#imports OpenAI
import openai
from dotenv import load_dotenv, find_dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 91.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 6.0 MB/s eta 0:00:00


In [4]:
_ = load_dotenv(find_dotenv()) # read local .env file

#openai.api_key = os.environ["OPENAI_API_KEY"]

#configure aqui sua apikey da openai
openai.api_key = 'SUA_API_KEY'

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def predict(text):
    prompt_text = f"""
        return 'negative' if the sentiment is negative or return 'positive' \
        if the sentiment is positive for the following text: '''{text}'''
        """
    response = get_completion(prompt_text)

    return response

    #prompt_text = f"""
    #Please keep {response} JSON format, but translate keys and values to portuguese from brazil.

    #No explanation is needed.
    #"""
    #response = get_completion(prompt_text)

In [ ]:
print(predict(str("Gostaria de saber o que houve cm a questão de conseguir olhar o caminho que o motoboy está percorrendo quando se inicia a nossa entrega!? Antes ficava mais fácil aguardar, pq víamos quando estava chegando, agora não aparece mais. É algum bug ou fizeram a retirada?").lower()))

negative


In [5]:
# Dataset class
class SentimentDataset(Dataset):
    def __init__(self, txt_list, label_list, tokenizer, max_length):
        # define variables
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        map_label = {0:'negative', 4: 'positive'}
        # iterate through the dataset
        for txt, label in zip(txt_list, label_list):
            # prepare the text
            prep_txt = f'<|startoftext|>Review: {txt}\nSentiment: {map_label[label]}<|endoftext|>'
            # tokenize
            encodings_dict = tokenizer(prep_txt, truncation=True,
                                       max_length=max_length, padding="max_length")
            # append to list
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
            self.labels.append(map_label[label])

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx], self.labels[idx]

# Data load function
def load_sentiment_dataset(tokenizer, random_seed = 1, file_path="/content/drive/My Drive/Colab/reviews_final.csv"):
    # load dataset and sample 10k reviews.
    #df = pd.read_csv(file_path, encoding='ISO-8859-1', header=None)
    #df = pd.read_csv(file_path, header=True, on_bad_lines='skip')
    df = pd.read_csv(file_path, header=None, on_bad_lines='skip')
    #df = df[[0, 5]]
    df = df[[0, 1]]
    df.columns = ['label', 'text']
    #df = df.fillna(0)
    #df['label'] = df['label'].astype(int)
    df = df.sample(7400, random_state=0)
    #df = df[df['text'].notna()]

    ##DEBUG
    #pd.set_option('display.max_rows', None)
    #print(df)



    def pick_first_n_words(string, max_words=250): # tried a few max_words, kept 250 as max tokens was < 512
        #print(string)
        split_str = string.split()
        #print(split_str[:min(len(split_str), max_words)])
        return " ".join(split_str[:min(len(split_str), max_words)])

    df['text'] = df['text'].apply(lambda x: pick_first_n_words(x))

    print("############")


    # divide into test and train
    X_train, X_test, y_train, y_test = \
              train_test_split(df['text'].tolist(), df['label'].tolist(),
              shuffle=True, test_size=0.05, random_state=random_seed, stratify=df['label'])

    # get max length
    max_length_train = max([len(tokenizer.encode(text)) for text in X_train])
    max_length_test = max([len(tokenizer.encode(text)) for text in X_test])
    max_length = max([max_length_train, max_length_test]) + 10  #for special tokens (sos and eos) and fillers
    max_length = max(max_length, 300)
    print(f"Setting max length as {max_length}")

    # format into SentimentDataset class
    train_dataset = SentimentDataset(X_train, y_train, tokenizer, max_length=max_length)

    # return
    return train_dataset, (X_test, y_test)

In [6]:
# import
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel

# model
model_name = "gpt2"
seed = 42

# seed
torch.manual_seed(seed)

In [7]:
# iterate for N trials
for trial_no in range(3):

  print("Loading model...")
  # load tokenizer and model
  tokenizer = GPT2Tokenizer.from_pretrained(model_name, bos_token='<|startoftext|>',
                                                eos_token='<|endoftext|>', pad_token='<|pad|>')
  model = GPT2LMHeadModel.from_pretrained(model_name).cuda()
  model.resize_token_embeddings(len(tokenizer))

  print("Loading dataset...")
  train_dataset, test_dataset = load_sentiment_dataset(tokenizer, trial_no)

    #print("Start training...")
    #training_args = TrainingArguments(output_dir='results', num_train_epochs=10,
                                    #logging_steps=10, load_best_model_at_end=True,
                                     # save_strategy="no", evaluation_strategy="no", per_device_train_batch_size=2, per_device_eval_batch_size=2,
                                    #warmup_steps=100, weight_decay=0.01, logging_dir='logs')

    #Trainer(model=model, args=training_args, train_dataset=train_dataset,
            #eval_dataset=test_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                                  #'attention_mask': torch.stack([f[1] for f in data]),
                                                                  #'labels': torch.stack([f[0] for f in data])}).train()

    # test
    #print("Start testing...")
    # eval mode on model
    #_ = model.eval()

Loading model...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading dataset...
############
Setting max length as 355
Loading model...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading dataset...
############
Setting max length as 355


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading model...
Loading dataset...
############
Setting max length as 355


In [8]:
predicted2 = []

In [13]:
#Realizando consultas ao modelo treinado da OpenAI:
for x in range(369, 370):
  try:
    res = predict(str(test_dataset[0][x]).lower())
  except:
    print("skipping at " + x)
  predicted2.insert(x, res)
  #predicted2.append(res)
  time.sleep(1)
  print(x)


369


In [14]:
len(predicted2)

370

In [15]:
print(test_dataset[0][369])
print(predicted2[369])


Muito mais intuitivo que os mais famosos; bem brasileiro/ carioca no linguajar. Gosto muito.
positive


In [17]:
#Calculando F1 score:
#F1 score = 2 * (precisão * revocação) / (precisão + revocação)
#Precisão = verdadeiros positivos / (verdadeiros positivos + falsos positivos)
#Revocação = verdadeiros positivos / (verdadeiros positivos + falsos negativos)

i = 0
verdadeiros_positivos = 0
verdadeiros_negativos = 0
falsos_positivos = 0
falsos_negativos = 0

for x in range(370):
  #print(x)
  if (test_dataset[1][x] == 4 and predicted2[x] == 'positive'):
    verdadeiros_positivos = verdadeiros_positivos + 1
  if (test_dataset[1][x] == 0 and predicted2[x] == 'negative'):
    verdadeiros_negativos = verdadeiros_negativos + 1
  if (test_dataset[1][x] == 4 and predicted2[x] != 'positive'):
    falsos_positivos = falsos_positivos + 1
  if (test_dataset[1][x] == 0 and predicted2[x] != 'negative'):
    falsos_negativos = falsos_negativos + 1
    i = i + 1
    #print(x)

precisao = verdadeiros_positivos / (verdadeiros_positivos + falsos_positivos)
revocacao = verdadeiros_positivos / (verdadeiros_positivos + falsos_negativos)
F1_score = 2 * (precisao * revocacao) / (precisao + revocacao)

print(F1_score)

#aproveitamento = 100 - ((100*i)/370)
#print("\n\nHouve " + str(i) + " difenças..\nEquivalente a um aproveitamento\
# de " + str(aproveitamento) + "%")

0.8753462603878117
